#### Merging intervals


About the pattern
The merge intervals pattern deals with problems involving overlapping intervals. Each interval is represented by a start and an end time. For example, an interval of [10,20] start at 10 and end at 20. This pattern involves tasks such as merging intersecting intervals, inserting new intervals into existing sets, or determining the minimum number of intervals needed to cover a given range. The most common problems solved using this pattern are event scheduling, resource allocation, and time slot consolidation.

The key to understanding this pattern and exploiting its power lies in understanding how any two intervals may overlap. The illustration below shows different ways in which two intervals can relate to each other:

e.g. 
* [1,4], [3,7] -> [1,7]
* [1,4], [3,7], [9,12] -> [1,7], [9,12]

Application:
1. Merge intervals: Given a sorted list of intervals, merge all overlapping intervals.
2. Meeting rooms: Given an array of meeting time intervals consisting of start and end times, determine if a person could attend all meetings.


#### Q1
We are given an array of closed intervals, intervals, where each interval has a start time and an end time. The input array is sorted with respect to the start times of each interval. For example, intervals = 
[ [1,4], [3,6], [7,9] ]
 is sorted in terms of start times 1, 3, and 7.

Your task is to merge the overlapping intervals and return a new output array consisting of only the non-overlapping intervals.

e.g. [[1,5],[3,7],[4,6],[6,8]] are overlapping -> merging [1,8]



In this solution, we use the merge intervals pattern with a simple linear scan to merge the overlapping intervals. First, we create an output list and copy the first interval of the input list to it. Next, we traverse the remaining intervals of the input list and check whether any interval overlaps with the interval present in the output list. If they overlap, update the interval in the output list. Otherwise, add the current input interval to the output list. Repeat this for all the intervals in the input list. Please note that when we have more than one interval in the output list, we compare the input intervals with the last interval of the output list.

* Inside the loop, we check each interval of the input list against the last interval of the output list. For each interval in the input list, we do the following:

* If the current input interval is overlapping with the last interval in the output list, we merge these two intervals and replace the last interval of the output list with the newly merged interval.
Otherwise, we add the input interval to the output list.

To check if the current input interval and the last interval in the output list overlap, we’ll check the start time of the current interval and the end time of the last interval in the output list. If the start time of the current interval is less than the end time of the last interval in the output list, that is, curr_start <=> prev_end, the two intervals overlap. Otherwise, they don’t overlap. Since the intervals are sorted in terms of their start times, we won’t encounter cases where the current interval’s start and end times are less than the start time of the last interval in the output list.

In [ ]:
# optimized solution
def merge_intervals(intervals):
    interval_merge = []

    for element in intervals:
        
        if not interval_merge:
            interval_merge.append(element)
            continue
        
        element_small = element[0] # element_small > interval
        element_large = element[1]

        interval_merge_len = len(interval_merge)
        
        # traverse interval merge list
        idx_insert = 0

        # comparing interval_merge from right to left
        # only need to compare last element
        last_min = interval_merge[interval_merge_len-1][0]
        last_max = interval_merge[interval_merge_len-1][1]
        if element_small > last_max:
            interval_merge.append(element)
        elif element_large > last_max:
            interval_merge[interval_merge_len-1][1] = element_large
        else:
            continue
    return interval_merge